# Total Network Objects

## Counting the number of cell towers per region

In this worked example we use FlowClient to count the number of cell towers per region. This is designed as a simple example to demonstrate that a FlowKit system has been successfully deployed.

The Jupyter notebook for this worked example can be downloaded [here](https://github.com/Flowminder/FlowKit/raw/master/docs/source/worked_examples/total-network-objects.ipynb), or can be run using the [quick start setup](../install.md#quickinstall).

### Load FlowClient and connect to FlowAPI

We start by importing FlowClient. We also import [geopandas](http://geopandas.org/) and [mapboxgl](https://mapbox-mapboxgl-jupyter.readthedocs-hosted.com/en/latest/), which we will use later to to visualise the data.

Mapbox requires an [access token](https://www.mapbox.com/account/), which should be set as the environment variable `MAPBOX_ACCESS_TOKEN`.

In [ ]:
import flowclient
import os
import numpy as np
import geopandas as gpd
import mapboxgl
from mapboxgl.utils import create_color_stops

mapbox_token = os.environ["MAPBOX_ACCESS_TOKEN"]

We must next generate a FlowAPI access token using [FlowAuth](../analyst.md#flowauth), and paste the token here as `TOKEN`. Once we have a token, we can start a connection to the FlowAPI system.

In [ ]:
conn = flowclient.connect(
    url=os.getenv("FLOWAPI_URL", "http://localhost:9090"), token=TOKEN
)

### Get tower counts

We can get cell tower counts using a `total_network_objects` query. We start by creating a specification for a `total_network_objects` query to count towers per level 3 administrative unit during the first week of 2016.

In [ ]:
query_spec = flowclient.total_network_objects(
    start_date="2016-01-01",
    end_date="2016-01-08",
    aggregation_unit="admin3",
    total_by="month"
)
query_spec

We run this query using `get_result`, which returns the result as a pandas `DataFrame`.

In [ ]:
towers_per_admin3 = flowclient.get_result(connection=conn, query=query_spec)
towers_per_admin3.head()

### Visualise tower counts on a choropleth map

We use the `get_geography` function to download the geography for the level 3 administrative regions as GeoJSON.

In [ ]:
# Download geography data as GeoJSON.
regions = flowclient.get_geography(connection=conn, aggregation_unit="admin3")

# Create a geopandas GeoDataFrame from the GeoJSON
regions_geodataframe = gpd.GeoDataFrame.from_features(regions)

We can now combine the result of the `total_network_objects` query with the geography data, and use the `Mapbox GL` library to create a choropleth showing the distribution of cell towers.

In [ ]:
towers_per_admin3_geodataframe = regions_geodataframe.join(
        towers_per_admin3
        .set_index("pcod"),
        on="admin3pcod",
        how="left",
    ).fillna(0).drop(columns=["centroid", "datetime"]).rename(
    columns={"admin3pcod": "P-code", "admin3name": "Name", "value": "Number of towers"}
)

In [ ]:
modal_locations_viz = mapboxgl.ChoroplethViz(
    towers_per_admin3_geodataframe.__geo_interface__,
    access_token=mapbox_token,
    color_property="Number of towers",
    color_stops=create_color_stops(
        np.linspace(1, towers_per_admin3_geodataframe["Number of towers"].max(), 9), colors="YlGn"
    ),
    opacity=0.8,
    line_color="black",
    line_width=0.5,
    legend_gradient=True,
    legend_layout="horizontal",
    legend_text_numeric_precision=0,
    below_layer="waterway-label",
    center=(84.1, 28.4),
    zoom=5.5,
)

modal_locations_viz.show()